In [ ]:
# !pip install comfit -q # Uncomment and run this cell if you are using Google Colab

If you find any errors in this tutorial, please raise a issue at [GitHub](https://github.com/vidarsko/ComFiT/issues).

# Phase-field crystal tutorail: Polycrystalline materials

In this tutorial, we will look at different ways of creating polycrystalline materials with the PFC framework.

The first way to create a polycrystalline material is to quench the PFC. 

In [ ]:
import comfit as cf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt


In [ ]:
pfc = cf.PhaseFieldCrystal2DTriangular(30, 23, plot_lib='matplotlib')

#Set a random initial condition
pfc.psi = 0.01*(np.random.rand(pfc.xRes,pfc.yRes) - 0.5) + pfc.psi0
pfc.psi_f = sp.fft.fftn(pfc.psi)

pfc.plot_field(pfc.psi)

In [ ]:
pfc.evolve_PFC(3000)
pfc.plot_field(pfc.psi)

Increasing the size of the simulation will create a larger polycrystalline material.

However, in many situations we want to control the polycrystalline material. 
In this case, we want to prescribe the regions with the different lattice orientations. 
Let's see how that can be done

We start by creating a simple inclusion, which is rotated with respect to the rest of the material.
To easily see what is going on, we will use the square PFC model.

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(40, 40, plot_lib='matplotlib')

# This creates a standard orientation of the crystal
pfc.conf_PFC_from_amplitudes()


psi_rotated = pfc.calc_PFC_from_amplitudes(rotation=np.pi/3)
pfc.plot_field(psi_rotated)


Now we just need to paste this region in the center onto the old field contained in the variable `pfc.psi`.

In [ ]:
# Specify the region centered at the mid position with radius 5 a0.
inclusion_region = pfc.calc_region_disk(pfc.rmid, 5*pfc.a0)

# Set the rotated field in the inclusion region
pfc.psi[inclusion_region] = psi_rotated[inclusion_region]
pfc.psi_f = sp.fft.fftn(pfc.psi)

pfc.plot_field(pfc.psi)

The interface is very sharp. In order to smooth it, we simply evolve the field for a few time steps.

NB! Remember that in order for this method to work, we need to also specify the fourier transform of the field, which is why we have added the `pfc.psi_f = sp.fft.fftn(pfc.psi)` line above. 

In [ ]:
pfc.evolve_PFC(100)

pfc.plot_field(pfc.psi)

Let create a bigger inclusion and create an evolution gif. 

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(40, 40, plot_lib='matplotlib')

# This creates a standard orientation of the crystal
pfc.conf_PFC_from_amplitudes()

# Create the rotated field
psi_rotated = pfc.calc_PFC_from_amplitudes(rotation=np.pi/6)

# Specify the region centered at the mid position with radius 5 a0.
inclusion_region = pfc.calc_region_disk(pfc.rmid, 10*pfc.a0)

# Set the rotated field in the inclusion region
pfc.psi[inclusion_region] = psi_rotated[inclusion_region]
pfc.psi_f = sp.fft.fftn(pfc.psi)

for n in range(100):
    pfc.evolve_PFC(100)
    fig = pfc.plot_field(pfc.psi)
    pfc.plot_save(fig, n)
cf.tool_make_animation_gif(n,'Inclusion_PFC_normal')

Now, the standard PFC evolution does not make the inclusion shrink by any meaningful amount. 
In order to see some more pronounced evolution, we will look at the same simulation, but now evolving according to the sHPFC model.
The sHPFC model will initiate a velocity field with the PFC that are saved in the components `psi[1]` and `psi[2]`, while the PFC field itself will be saved in `psi[0]`, which needs to be taken into account when plotting the field. 
Also, it is important to smooth the interface by evolving the field according to normal PFC dynamics for a few time steps before starting the sHPFC evolution. 
This is done in the following script


In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(40, 40, plot_lib='matplotlib')

# This creates a standard orientation of the crystal
pfc.conf_PFC_from_amplitudes()

# Create the rotated field
psi_rotated = pfc.calc_PFC_from_amplitudes(rotation=np.pi/6)

# Specify the region centered at the mid position with radius 5 a0.
inclusion_region = pfc.calc_region_disk(pfc.rmid, 10*pfc.a0)

# Set the rotated field in the inclusion region
pfc.psi[inclusion_region] = psi_rotated[inclusion_region]
pfc.psi_f = sp.fft.fftn(pfc.psi)

# Smooth the interface
pfc.evolve_PFC(100)

for n in range(100):
    pfc.evolve_PFC_hydrodynamic(100, rho0=2**-6, gamma_S=2**-6)
    fig = pfc.plot_field(pfc.psi[0])
    pfc.plot_save(fig, n)
cf.tool_make_animation_gif(n,'Inclusion_PFC_sHPFC')

As we see, the presence of the velocity field makes the inclusion shrink faster. 

The inclusion is one of the types of preconfigured polycrystalline materials that can be created with the PFC framework.
Another type is the four-grain configuration used in the paper [Hydrodynamic phase field crystal approach to interfaces, dislocations and multi-grain networks](http://iopscience.iop.org/article/10.1088/1361-651X/ac9493).
Both of these configurations are available through the `conf_create_polycrystal` method in the pfc class. 
In this method, `relaxation_time` is an optional keyword specifying for how long a simulation time the initial polycrystalline edges are evolved according to classical PFC dynamics to smooth them before running. 
More information on these methods can be found in the documentation.

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(30,30, plot_lib='matplotlib')

# This creates a standard orientation of the crystal
pfc.conf_create_polycrystal(type='circular', relaxation_time=10)
pfc.plot_field(pfc.psi)

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(30,30, plot_lib='matplotlib')

# This creates a standard orientation of the crystal
pfc.conf_create_polycrystal(type='four_grain', relaxation_time=10)
pfc.plot_field(pfc.psi)
